In [22]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import shutil
import pyperclip
import time
import pytz
import datetime
import os
from tqdm import tqdm
import random

In [23]:
def driver_setup():# Assigning a Headless Firefox Driver
    options = webdriver.FirefoxOptions()
    options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
    options.add_argument('E:/Programs/geckodriver.exe')
    # options.add_argument('--headless')
    options.add_argument("--start-with-cache")
    options.add_argument("0")
    options.add_argument("--disk-cache-size=0")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    return driver

def save_data_to_disk(data, filename):
    max_attempts = 5  # Maximum number of attempts
    wait_time = 5  # Time to wait between attempts (in seconds)
    
    for _ in range(max_attempts):
        try:
            backup_filename = filename + 'bak.parquet'
            if os.path.isfile(backup_filename):
                os.remove(backup_filename)  # Remove old backup before creating a new one
            shutil.copyfile(filename, backup_filename)
            
            if os.path.isfile(filename):
                existing_df = pd.read_parquet(backup_filename)
                data = pd.concat([existing_df, data], ignore_index=True)
                data.to_parquet(filename, index=False)
            else:
                data.to_parquet(filename, index=False)
            break  # Break out of the loop if successful
        except Exception as e:
            print(f"Error: {e}")
            print(f"File '{filename}' is in use. Waiting {wait_time} seconds before trying again...")
            time.sleep(wait_time)
    else:
        print(f"Failed to save data to '{filename}' after {max_attempts} attempts.")

In [24]:
driver = driver_setup()

The version of firefox cannot be detected. Trying with latest driver version


In [25]:
driver.get(f"https://www.linkedin.com/authwall?trk=qf&original_referer=&sessionRedirect=https%3A%2F%2Fwww.linkedin.com%2F")
Sign_in_button = driver.find_element(By.XPATH, "/html/body/div/main/div/form/p/button").click()
time.sleep(5)
email_phone = driver.find_element(By.XPATH, '//*[@id="session_key"]')
password = driver.find_element(By.XPATH, '//*[@id="session_password"]')
email_phone.send_keys("pannersmail@gmail.com")
# email_phone.send_keys("urvibabla@gmail.com")
# email_phone.send_keys("r.panneerselvam@firebird.net.in")
password.send_keys("MK90473@yahoo.com")
# password.send_keys("SkillIQ@123")

# email_phone.send_keys("chakrabortiarjun@gmail.com")
# password.send_keys("Reafactor_Academy")

# email_phone.send_keys("renjithcm.renju@gmail.com")
# password.send_keys("Renjith#@1")
submit_button = driver.find_element(By.XPATH, "/html/body/div/main/div/div/form[1]/div[2]/button")
submit_button.click()

wait = WebDriverWait(driver, 4)

In [26]:
driver.switch_to.window(driver.window_handles[0])

# Sales Navigator

# connection reqs extract

In [10]:
dalis = []

In [ ]:
# scroll to view implimentation.
run = True
page = 0
while run == True:
    page += 1
    driver.execute_script("window.scrollTo(0, 0);")
    nextorli = 1
    time.sleep(2)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')))
    li_elements = driver.find_elements(By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')
    for li in range(len(li_elements)):
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        except StaleElementReferenceException:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        dalis.append({'textcol':li_elements[li].text})
        
        print(f'\rLi: {li} lent: {len(li_elements)}, page: {page} tot_extract: {len(dalis)}', end='')
    time.sleep(1)
    next_button = driver.find_element(By.CSS_SELECTOR, '.artdeco-pagination__button.artdeco-pagination__button--next.artdeco-button.artdeco-button--muted.artdeco-button--icon-right.artdeco-button--1.artdeco-button--tertiary.ember-view')
    
    if next_button.is_enabled():
        driver.find_element(By.XPATH, '//button[@aria-label="Next"]').click()
        time.sleep(2)
    else:
        run = False

In [38]:
pd.DataFrame(dalis).to_csv('connection_req_proper_HR_names.csv',index=False)

# Contact extraction

In [6]:
contalis = []

In [18]:
# scroll to view implimentation.
run = True
page = 0
while run == True:
    page += 1
    driver.execute_script("window.scrollTo(0, 0);")
    nextorli = 1
    time.sleep(2)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')))
    li_elements = driver.find_elements(By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')
    for li in range(len(li_elements)):
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        except StaleElementReferenceException:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        # time.sleep(1)
        # clicking the name so profile opens.
        if nextorli:
            WebDriverWait(li_elements[li], 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".artdeco-entity-lockup__content.ember-view a"))).click()
            nextorli = 0
        else:
            driver.find_elements(By.CSS_SELECTOR, '._inline-sidesheet-header_1cn7lg > div:nth-child(1) > button')[1].click()
        time.sleep(1)
        try:
            phonetext = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "._layout_hqxetg._panel_hqxetg._grid_17ha7o._gap-none_17ha7o"))).text
        except:
            phonetext = ''
        
        if 'phone' in phonetext:
            
            data = {}
            
            name_and_comp = WebDriverWait(li_elements[li], 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".artdeco-entity-lockup__content.ember-view a")))
            namelis = ['name','comp']
            for i in range(len(name_and_comp)):
                data[namelis[i]] = name_and_comp[i].text
            #------------------------------------------------------------------------------------------------------    
            try:
                title = driver.find_element(By.CSS_SELECTOR, '.lbhdzEDRUBChRxKDPXPcbnKQXXAJiQHiDW._inset-horizontal-compact_sfmhx2 > div > div').text
            except:
                title = driver.find_element(By.CSS_SELECTOR, '.lbhdzEDRUBChRxKDPXPcbnKQXXAJiQHiDW._inset-horizontal-compact_sfmhx2 ul > li').text
            data['title'] = title   
            #------------------------------------------------------------------------------------------------------
            driver.find_element(By.CSS_SELECTOR, '._layout_hqxetg._panel_hqxetg._grid_17ha7o._gap-none_17ha7o button').click()
            time.sleep(1)
            sections = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".artdeco-modal__content.ember-view section")))
            for section in sections:
                head = section.find_element(By.CSS_SELECTOR, 'h3').text
                try:
                    tail = section.find_element(By.CSS_SELECTOR, 'div').text.split('\n')[0]
                except:
                    tail = 'NA'
                data[head]=tail
            
            driver.find_elements(By.CSS_SELECTOR, '.artdeco-modal.artdeco-modal--layer-default  button')[0].click() #close button
            time.sleep(1)
            
            contalis.append(data)
        print(f'\rLi: {li} lent: {len(li_elements)}, page: {page} tot_extract: {len(contalis)}', end='')
    
    next_button = driver.find_element(By.CSS_SELECTOR, '.artdeco-pagination__button.artdeco-pagination__button--next.artdeco-button.artdeco-button--muted.artdeco-button--icon-right.artdeco-button--1.artdeco-button--tertiary.ember-view')
    
    if next_button.is_enabled():
        driver.find_element(By.XPATH, '//button[@aria-label="Next"]').click()
        time.sleep(2)
    else:
        run = False

Li: 5 lent: 6, page: 12 tot_extract: 10301

In [19]:
df = pd.DataFrame(contalis)

In [21]:
df.rename(columns= {'name':'Name', 'Email address':'Email','comp':'Company/Organization', 'title':'Position/Role'},inplace=True)

In [27]:
exisdf = pd.read_parquet('original_syeda_contacts.parquet')
toparkdf = pd.concat([exisdf,df],ignore_index=True)
toparkdf.drop_duplicates(inplace=True)
toparkdf = toparkdf.reset_index(drop=True)
toparkdf.to_parquet('original_syeda_contacts.parquet',index=False)

In [28]:
toparkdf.reindex(columns=['Name','Email','Phone','Company/Organization','Position/Role']).to_excel('linkedincontacts.xlsx',index=False)

# connection from sales navigator.      

In [27]:
# scroll to view implimentation.
run = True
page = 0
while run == True:
    page += 1
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')))
    li_elements = driver.find_elements(By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')
    for li in range(len(li_elements)):
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        except StaleElementReferenceException:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        # time.sleep(1)
        
        WebDriverWait(li_elements[li], 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".mt4.mr5 li")))[0].click()
        try:
            buttons3 = WebDriverWait(driver, 3).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "._container_x5gf48._container_iq15dg._raised_1aegh9.ph2 button")))
        except:
            WebDriverWait(li_elements[li], 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".mt4.mr5 li")))[0].click()
            buttons3 = WebDriverWait(driver, 3).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "._container_x5gf48._container_iq15dg._raised_1aegh9.ph2 button")))
        for buttons in buttons3:
            if buttons.text == 'Connect':
                buttons.click()
                time.sleep(2)
                if 'email is required to connect' in driver.find_element(By.CSS_SELECTOR,'.connect-cta-form__content-container').text:
                    driver.find_element(By.CSS_SELECTOR,'.artdeco-modal__dismiss.artdeco-button.artdeco-button--circle.artdeco-button--muted.artdeco-button--2.artdeco-button--tertiary.ember-view').click()
                    time.sleep(1)
                    continue
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".button-primary-medium.connect-cta-form__send"))).click()
                time.sleep(2)
            else:
                WebDriverWait(li_elements[li], 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".mt4.mr5 li")))[0].click()

        print(f'\rLi: {li} lent: {len(li_elements)}, page = {page}', end='')
        li += 1
    try:
        driver.find_element(By.XPATH, '//button[@aria-label="Next"]').click()
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        run = False

Li: 0 lent: 25, page = 32

ElementClickInterceptedException: Message: Element <li> is not clickable at point (1746,215) because another element <div id="ember10930" class="artdeco-modal-overlay artdeco-modal-overlay--layer-default artdeco-modal-overlay--is-top-layer  ember-view"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:333:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:176:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:135:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:84:31


In [25]:
for i in driver.find_elements(By.CSS_SELECTOR, '._container_x5gf48._container_iq15dg._raised_1aegh9.ph2 button'):
    print(i.text)

Connect — Pending
Edit list
Unsave


# Sending messages from sales navigator

In [ ]:
message = """Hi,

I hope this message finds you well. We are excited to invite you to a webinar on Saturday 4:00pm, 6th April 2024, where we will introduce PayPulse - our cutting-edge tool for real-time salary data and compensation insights.

Key Benefits of PayPulse:
- Empower your HR team with accurate salary benchmarks.
- Real Time insights tailored to your industry and location.
- Weekly updated data from 25,000 companies and 150,000 jobs.

Don't miss this opportunity to revolutionize your compensation strategy with PayPulse. 

Register here: https://forms.gle/mCgg6sZsEL2ZHEfZ8

Looking forward to having you join us for this informative session!

Check Our Company Page: 'https://www.linkedin.com/company/equipay-partners'
Check Our website: 'https://equipaypartners.com/'

Please reply unsubscribe if you don’t want to hear from us."""

message2 = """Thank you for your interest in attending our webinar! We're glad to hear that you're considering joining.

The webinar promises to be an insightful session, and we believe you'll find the information valuable.

kindly register in the link below for the webinar
https://forms.gle/mCgg6sZsEL2ZHEfZ8
"""

In [ ]:
# scroll to view implimentation.
run = True
page = 0
while run == True:
    page += 1
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')))
    li_elements = driver.find_elements(By.XPATH, '//ol[contains(@class, "artdeco-list")]/li')
    for li in range(len(li_elements)):
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        except StaleElementReferenceException:
            driver.execute_script("arguments[0].scrollIntoView(true);", li_elements[li])
        # time.sleep(1)
        try:
            [butele.click() for butele in WebDriverWait(li_elements[li], 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".mt4.mr5 li"))) if 'Message' in butele.text]
        except:
            [butele.click() for butele in WebDriverWait(li_elements[li], 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".mt4.mr5 li"))) if 'Message' in butele.text]
        time.sleep(2)
        message_box = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.compose-form__message-field.pv3.flex-grow-1.flex-shrink-zero.border-none.resize-none')))
        message_box.clear()
        content = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.message-container-width.flex.flex-column.full-height.overflow-y-auto'))).text.split('\n')[-1]

        if 'beginning of your conversation' in content:
            message_box.send_keys(message)
            #sendbutton
            driver.find_element(By.CSS_SELECTOR,'.ember-view._button_ps32ck._small_ps32ck._primary_ps32ck._left_ps32ck._container_iq15dg.ml4').click()
            time.sleep(1)
        elif 'thank' in content.lower():
            message_box.send_keys(message2)
            #sendbutton
            driver.find_element(By.CSS_SELECTOR,'.ember-view._button_ps32ck._small_ps32ck._primary_ps32ck._left_ps32ck._container_iq15dg.ml4').click()
            time.sleep(1)
        
        WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,'.z-index-1.m0.pv1.pl3.background-color-blue9.elevation-2dp.flex.flex-shrink-zero.align-items-center button')))[-1].click()
        time.sleep(1)
        print(f'\rLi: {li} lent: {len(li_elements)}, page = {page}', end='')
        li += 1
        
    try:
        driver.find_element(By.XPATH, '//button[@aria-label="Next"]').click()
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)
    except NoSuchElementException:
        run = False

# SALES Navigator Message intrest finding. and also Arjun lead table extraction.

In [ ]:
while True:
    last_li_element = driver.find_elements(By.CSS_SELECTOR, '.overflow-y-auto.overflow-hidden.flex-grow-1 > ul li')[-1]
    driver.execute_script("arguments[0].scrollIntoView();", last_li_element)
    try:
        button = driver.find_element(By.CSS_SELECTOR, '.overflow-y-auto.overflow-hidden.flex-grow-1 > div button')
        driver.execute_script("arguments[0].scrollIntoView();", button)
        button.click()
    except:
        None
    time.sleep(2)

In [ ]:
len(driver.find_elements(By.CSS_SELECTOR, '.overflow-y-auto.overflow-hidden.flex-grow-1 > ul li'))

In [ ]:
st_ran = 0

In [ ]:
li_ele = driver.find_elements(By.CSS_SELECTOR, '.overflow-y-auto.overflow-hidden.flex-grow-1 > ul li')
allrows = []
for li_ind in range(st_ran,len(li_ele)):    
    data = {}
    
    last_message = li_ele[li_ind].find_element(By.CSS_SELECTOR,'.t-14.t-black--light.truncated-content.break-words').text
    if not 'Hi, I hope this message finds you well.' in last_message:
        li_ele[li_ind].find_element(By.CSS_SELECTOR,'a').click()
        time.sleep(2)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.flex.align-items-flex-start a')))
        data['name'] = driver.find_element(By.CSS_SELECTOR, '.flex.align-items-flex-start a').find_element(By.CSS_SELECTOR, '.artdeco-entity-lockup__title.ember-view').text
        data['sales_nav_link'] = driver.find_element(By.CSS_SELECTOR, '.flex.align-items-flex-start a').get_attribute('href')
        data['messages'] = driver.find_element(By.CSS_SELECTOR,'.message-container-width.flex.flex-column.full-height.overflow-y-auto').text
        allrows.append(data)
    print(f'\rcompletion {li_ind}')
    st_ran = li_ind+1

In [ ]:
pd.DataFrame(allrows).to_excel('message_responders.xlsx',index=False)

In [ ]:
df = pd.DataFrame()

In [ ]:
# leads data extraction
while True:    
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.background-color-white.models-table-wrapper.ember-view table')))
    table = driver.find_element(By.CSS_SELECTOR, '.background-color-white.models-table-wrapper.ember-view table')
    html = table.get_attribute('outerHTML')
    tempdf = pd.read_html(html)[0]
    trow = table.find_elements(By.CSS_SELECTOR,'tr')[1:]
    profilelinks = []
    for i in trow:
        profilelinks.append(i.find_element(By.CSS_SELECTOR,'td a').get_attribute('href'))
    tempdf['slaesnav_links'] = profilelinks
    df = pd.concat([df,tempdf],ignore_index=True)
    try:
        next = driver.find_elements(By.CSS_SELECTOR,'.artdeco-models-table-pagination__pagination-cmpt.artdeco-pagination.artdeco-pagination--has-controls.ember-view > button')[1]
        driver.execute_script("arguments[0].scrollIntoView();", next)
        next.click()
        time.sleep(2)
    except:
        break

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df

In [ ]:
df = pd.DataFrame(allrows)

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
st = 0

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
# linkedin url
for inde,row in df.iloc[st:].iterrows():
    driver.get(row['slaesnav_links'])
    # 3button
    wait = WebDriverWait(driver, 30)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.ember-view._button_ps32ck._small_ps32ck._tertiary_ps32ck._circle_ps32ck._container_iq15dg._overflow-menu--trigger_1xow7n')))
    driver.find_element(By.CSS_SELECTOR,'.ember-view._button_ps32ck._small_ps32ck._tertiary_ps32ck._circle_ps32ck._container_iq15dg._overflow-menu--trigger_1xow7n').click()
    for i in driver.find_elements(By.CSS_SELECTOR,'._container_x5gf48._visible_x5gf48._container_iq15dg._raised_1aegh9 > ul > li'):
        try:    
            if 'View LinkedIn profile' in i.text:
                try:
                    time.sleep(0.5)
                    i.click()
                except:
                    None
        except:
            None
    time.sleep(2)
    try:
        driver.switch_to.window(driver.window_handles[1])
    except:
        time.sleep(3)
        driver.switch_to.window(driver.window_handles[1])
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.pvs-header__left-container--stack')))
    df.at[inde,'linkedin_url'] = driver.current_url
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    print(f'\rcompletion {inde} total {len(df)}')
    st = inde+1

In [ ]:
df['Name'] = df['Name'].str.replace('Select LinkedIn Member  ','')

In [ ]:
df['title'] = df['Name'].apply(lambda x: x.split('Saved Badge')[1])

In [ ]:
for ind,row in df.iterrows():
    firstname = row['Name'].split(' ')[0]
    index = row['Name'].find(firstname,1)
    df.at[ind,'Name'] = row['Name'][:index]

In [ ]:
df

In [ ]:
df.to_excel('arjun_sir.xlsx',index=False)

In [ ]:
df.to_excel('message_responders.xlsx',index=False)

# END